In [16]:
import numpy as np
import pandas as pd
import seaborn as sns
import geopandas as gpd
from shapely.geometry import Point, Polygon
from geopandas import GeoDataFrame
import matplotlib.pyplot as plt
import sklearn
import sklearn.utils
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import DBSCAN
import matplotlib.pyplot as plt
import scipy.stats as stats
from datetime import datetime
import datetime
from datetime import timedelta

In [17]:
"""
Importing csv file with new data and filtering it further.

    Returns:
    _ _ _ _ _ _ _ _ _ _ _ _ _
    Dataframe with dropped nan values and dropped index.
    
"""
newdata = pd.read_csv("/work/Data/newdataset.csv")
newdata = newdata.dropna().drop(columns=["Unnamed: 0"])
newdata

,vin,sequence,first,last,start_lat,start_long,latitude,longitude,dwell time duration (seconds)
2,005b420fd9275f080ad79e1c4330357e,1143,1.578927e+09,1.578928e+09,41.267,-81.378,41.410,-81.500,8938.025
3,005b420fd9275f080ad79e1c4330357e,1145,1.578938e+09,1.578939e+09,41.394,-81.413,41.414,-81.504,10074.972
4,005b420fd9275f080ad79e1c4330357e,1146,1.578953e+09,1.578954e+09,41.409,-81.497,41.379,-81.427,14185.917
6,008beb5671fb49a92dc0ef9de96979bf,555,1.578443e+09,1.578445e+09,40.100,-83.166,40.149,-82.904,45104.446
7,008beb5671fb49a92dc0ef9de96979bf,556,1.578482e+09,1.578484e+09,40.149,-82.904,40.100,-83.166,37768.558
...,...,...,...,...,...,...,...,...,...
48212,ffffe9555b16c2d737f6f2e8616b1e68,644,1.578858e+09,1.578860e+09,40.178,-82.997,39.943,-82.827,4264.046
48213,ffffe9555b16c2d737f6f2e8616b1e68,645,1.578877e+09,1.578880e+09,39.942,-82.827,40.178,-82.997,16974.276
48214,ffffe9555b16c2d737f6f2e8616b1e68,646,1.578886e+09,1.578887e+09,40.178,-82.997,40.136,-82.996,6248.210
48215,ffffe9555b16c2d737f6f2e8616b1e68,647,1.578954e+09,1.578954e+09,40.136,-82.997,40.150,-82.997,67188.380


In [18]:
"""

Making sure there are no duplicated values in the dataframe.

"""
newdata[newdata.duplicated()]

,vin,sequence,first,last,start_lat,start_long,latitude,longitude,dwell time duration (seconds)


In [10]:
"""

Converting the timestamp column in the dataframe from milliseconds into seconds.

Making a new column in the dataframe with the converted timestamp column as a datetime object.

"""

newdata['timestamp'] = newdata['timestamp']/1000
newdata['datetime'] = pd.to_datetime(newdata['timestamp'], unit = 's')

KeyError: 'timestamp'

In [ ]:
## Getting first and last locations based on trip_sequence(engine cycle)
datetimeFormat = '%Y-%m-%d %H:%M:%S.%f'
timeline_df = newdata.groupby("sequence")["datetime"].agg(["first","last"])
timeline_df

## Getting the last latitude and longitude for each trip or sequence
timeline_df["latitude"] = newdata.groupby("sequence")["navigation_location_coordinate_latitude"].agg(["last"])
timeline_df["longitude"] = newdata.groupby("sequence")["navigation_location_coordinate_longitude"].agg(["last"])

x = timeline_df["last"]-timeline_df["first"]
duration = pd.to_timedelta(x.values, unit = 's').total_seconds()
timeline_df["trip_duration_s"] = duration

NameError: name 'df' is not defined

In [14]:
"""
Process filtered dataframe into a dataframe grouped by vin and sequence and aggregate by first 
and last to find the start latitude and longitudes and end latitudes and longitudes.

    Returns:
    _ _ _ _ _ _ _ _ _ _ _ _ _
    Dataframe grouped by vin and sequence with four new columns for start/end latitudes and longitudes.

"""
def gen_timeline_df(df):
    timeline = df.groupby(["vin","sequence"])["timestamp"].agg(["first","last"])
    timeline["start_lat"] = df.groupby(["vin","sequence"])["navigation_location_coordinate_latitude"].agg(["first"])
    timeline["start_long"] = df.groupby(["vin","sequence"])["navigation_location_coordinate_longitude"].agg(["first"])
    timeline["latitude"] = df.groupby(["vin","sequence"])["navigation_location_coordinate_latitude"].agg(["last"])
    timeline["longitude"] = df.groupby(["vin","sequence"])["navigation_location_coordinate_longitude"].agg(["last"])
    return timeline
"""
Process filtered dataframe into a dataframe with the dwell time duration for each completed trip
(time of previous trip location with engine off to time of current trip with engine on)
    
    Returns:
    _ _ _ _ _ _ _ _ _ _ _ _ _
    Dataframe with new column for dwell time duration in seconds.

"""
def calc_dwell_duration(df):
    df['dwell time duration (seconds)'] = ''
    vins = df.index.levels[0]
    for i in vins:
        num_trips = len(df.loc[i])
        for j in np.arange(num_trips):
            if j == 0:
                ## first trip has no associated dwell time
                df.loc[i].iloc[j,6] = ''
            else:
                ## find differences in timestamps between consecutive trips (dwell time)
                if df.loc[i].iloc[j, 0] - df.loc[i].iloc[j - 1, 1] < 0:
                    df.loc[i].iloc[j,6] = ''
                else:
                    df.loc[i].iloc[j,6] = df.loc[i].iloc[j, 0] - df.loc[i].iloc[j - 1, 1]
    


In [15]:
"""

Create a new dataframe with the original newdata dataframe using the two functions above to find 
the dwell time duration in seconds.

"""
dwell_df = gen_timeline_df(newdata)
calc_dwell_duration(dwell_df)
dwell_df.loc['123b9a7bbd7775662d3a55656dc2379b']

KeyError: 'Column not found: timestamp'

In [ ]:
"""

Convert final dataframe with all necessary columns into a csv file to import into other notebooks
for model implementation.

"""
dwell_df.to_csv("final_dwell_duration.csv", index = True,  float_format='%.3f' )

,Unnamed: 0.1,timestamp,vin,sequence,navigation_location_coordinate_latitude,navigation_location_coordinate_longitude,key,datetime
0,138.0,1.578942e+09,00089af9f7d3588a888379cf0ffc53cc,733,41.492962,-81.995163,eyJ0aW1lc3RhbXAiOjE1Nzg5NDIzMzM3MTQsInZpbiI6Ij...,2020-01-13 19:05:33.713999872
1,3548.0,1.578918e+09,005b420fd9275f080ad79e1c4330357e,1142,41.335046,-81.398117,eyJ0aW1lc3RhbXAiOjE1Nzg5MTc1MzU5NjMsInZpbiI6Ij...,2020-01-13 12:12:15.963000064
2,3549.0,1.578918e+09,005b420fd9275f080ad79e1c4330357e,1142,41.337986,-81.402637,eyJ0aW1lc3RhbXAiOjE1Nzg5MTc2NTk5NzQsInZpbiI6Ij...,2020-01-13 12:14:19.973999872
3,3550.0,1.578918e+09,005b420fd9275f080ad79e1c4330357e,1142,41.338722,-81.403610,eyJ0aW1lc3RhbXAiOjE1Nzg5MTc2Njg5NTUsInZpbiI6Ij...,2020-01-13 12:14:28.954999808
4,3556.0,1.578918e+09,005b420fd9275f080ad79e1c4330357e,1142,41.292217,-81.409466,eyJ0aW1lc3RhbXAiOjE1Nzg5MTgyMDY5NjAsInZpbiI6Ij...,2020-01-13 12:23:26.960000000
...,...,...,...,...,...,...,...,...
736059,235.0,1.580774e+09,cc63c9825a40f6dc03a7ccfc6cdc7e8d,623,40.089127,-82.838978,eyJ0aW1lc3RhbXAiOjE1ODA3NzQzOTg5ODcsInZpbiI6Im...,2020-02-03 23:59:58.987000064
736060,238.0,1.580774e+09,cc63c9825a40f6dc03a7ccfc6cdc7e8d,623,40.089245,-82.838651,eyJ0aW1lc3RhbXAiOjE1ODA3NzQzOTk5OTMsInZpbiI6Im...,2020-02-03 23:59:59.992999936
736061,241.0,1.580774e+09,d8538c269b6cb7eec9a1f3b2bd8fc2d7,764,40.304051,-83.524914,eyJ0aW1lc3RhbXAiOjE1ODA3NzQzOTg4NzgsInZpbiI6Im...,2020-02-03 23:59:58.878000128
736062,242.0,1.580774e+09,d8538c269b6cb7eec9a1f3b2bd8fc2d7,764,40.304008,-83.524871,eyJ0aW1lc3RhbXAiOjE1ODA3NzQzOTk4NzksInZpbiI6Im...,2020-02-03 23:59:59.879000064


<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=608d4940-9d52-4d53-8e1b-b72eb309e61f' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>